<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=45968854fa5bdddb7092ca13401b299897d25faaf7e0c95a717ee96c1c494f48
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 13:04:56
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.43 C
-------------------
Today PnL: 60.46 K (0.43%)
Current PnL: -16.54 L (-11.12%)
CY Booked + Current PnL: -5.51 L (-3.71%)
-------------------
Total profit:  2.35 L
Total loss:  -18.89 L
-------------------
Total Booked + Current PnL: 21.47 L (17.64%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.73%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.43 L (57.79%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-9.44,73.0,M-LC,4.35,223501.0,22022.0,5185.0,1.43,10.93,2.32,13.51,60.0,4.25,1.59,24.90,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.20,59.0,H-LC,10.97,173481.0,17304.0,8795.0,0.42,11.08,5.07,16.71,22.0,1.97,1.23,30.45,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,105.98,61.0,M-SC,4.26,88707.0,-12070.0,12162.0,-0.15,-11.98,13.71,0.09,245.0,-0.99,0.63,16.54,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,68.0,H-MC,6.31,114226.0,13722.0,16917.0,-0.44,13.65,14.81,30.49,79.0,0.81,0.81,50.48,MH,ATH,METALS
57,RELIANCE,1533.00,-11.79,59.0,H-LC,5.33,220334.0,9968.0,18817.0,0.51,4.74,8.54,13.68,37.0,0.53,1.57,22.04,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,42.0,H-MC,2.27,135826.0,38.0,42717.0,-0.45,0.03,31.45,31.48,107.0,0.0,0.97,12.15,X40,ATH,INSURANCE
52,PGHH,17973.08,-30.55,54.0,H-MC,4.29,200925.0,105.0,68676.0,0.19,0.05,34.18,34.25,80.0,0.0,1.43,5.25,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-15.68,44.0,H-LC,6.05,303688.0,-12154.0,124816.0,2.83,-3.85,41.10,35.67,5.0,-0.10,2.16,9.47,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.32,45.0,H-LC,7.97,218645.0,-33123.0,86343.0,0.15,-13.16,39.49,21.13,27.0,-0.38,1.56,17.61,X40,ATH,PAINTS
42,ITC,452.00,-37.92,49.0,H-LC,2.59,200392.0,254.0,20640.0,-0.80,0.13,10.30,10.44,4.0,0.01,1.43,5.70,X40,NTT,FMCG
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.85,213090.0,8590.0,32816.0,-0.23,4.20,15.40,20.25,10.0,0.26,1.52,13.30,X40N,BTT,PHARMA
44,JIOFIN,387.00,0.39,52.0,H-LC,13.32,272706.0,5341.0,60104.0,0.52,2.00,22.04,24.48,48.0,0.09,1.94,58.07,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-1.73,65.0,H-LC,1.07,243990.0,-2161.0,81371.0,0.87,-0.88,33.35,32.18,16.0,-0.03,1.74,34.85,X200,ATH,IT
42,ITC,452.0,-37.92,49.0,H-LC,2.59,200392.0,254.0,20640.0,-0.80,0.13,10.30,10.44,4.0,0.01,1.43,5.70,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.99,54.0,H-LC,4.06,280121.0,14695.0,42102.0,-0.59,5.54,15.03,21.40,11.0,0.35,1.99,13.02,XY25,NTT,FMCG
57,RELIANCE,1533.0,-11.79,59.0,H-LC,5.33,220334.0,9968.0,18817.0,0.51,4.74,8.54,13.68,37.0,0.53,1.57,22.04,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-20.12,50.0,H-LC,5.85,213090.0,8590.0,32816.0,-0.23,4.20,15.40,20.25,10.0,0.26,1.52,13.30,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-39.39,56.0,M-SC,40.35,60236.0,-4770.0,155698.0,3.50,-7.34,258.48,232.18,198.0,-0.03,0.43,4.90,XY24,NTT,MISC
84,VBL,671.64,-15.68,44.0,H-LC,6.05,303688.0,-12154.0,124816.0,2.83,-3.85,41.10,35.67,5.0,-0.10,2.16,9.47,X40N,ATH,FMCG
17,CAMPUS,393.00,-22.86,66.0,M-SC,1.35,160742.0,-7033.0,62882.0,2.76,-4.19,39.12,33.28,210.0,-0.11,1.14,24.59,XY24,NTT,FOOTWEAR
36,IEX,219.00,-28.31,61.0,H-SC,17.10,204744.0,4790.0,92442.0,2.37,2.40,45.15,48.63,115.0,0.05,1.46,14.23,XR,NTT,MISC
70,SURYODAY,240.00,44.74,57.0,H-SC,2.75,132631.0,-46440.0,112166.0,2.22,-25.93,84.57,36.71,135.0,-0.41,0.94,31.60,XR,NTT,BANKS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.57,84518.0,-29031.0,69085.0,-2.76,-25.57,81.74,35.27,268.0,-0.42,0.60,106.86,XR,NTT,HOTELS
37,INDIAMART,4850.92,-51.47,45.0,H-SC,12.00,135558.0,12222.0,121541.0,-1.54,9.91,89.66,108.45,119.0,0.10,0.96,34.48,AR,ATH,MISC
34,ICICIGI,2260.25,-18.95,50.0,H-MC,2.67,139409.0,3276.0,27854.0,-1.50,2.41,19.98,22.86,91.0,0.12,0.99,15.32,X40,ATH,INSURANCE
55,RAJOOENG,143.33,-35.29,41.0,H-SC,11.08,98140.0,-4360.0,45193.0,-1.39,-4.25,46.05,39.83,114.0,-0.10,0.70,2.58,AR,ATH,MISC
8,ATULAUTO,844.00,3856.76,63.0,M-SC,6.15,123357.0,-24011.0,75828.0,-1.36,-16.29,61.47,35.16,236.0,-0.32,0.88,26.00,XY24,NTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.98,61.0,M-SC,4.26,88707.0,-12070.0,12162.0,-0.15,-11.98,13.71,0.09,245.0,-0.99,0.63,16.54,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,55.0,H-SC,12.36,123795.0,-18684.0,38203.0,-0.26,-13.11,30.86,13.70,163.0,-0.49,0.88,46.11,OX40N,NTT,BANKS
19,CERA,9475.0,-19.81,48.0,H-SC,2.14,146314.0,-29589.0,71606.0,0.35,-16.82,48.94,23.89,149.0,-0.41,1.04,26.36,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.42,60.0,H-SC,1.70,224100.0,-45567.0,81909.0,1.60,-16.90,36.55,13.47,138.0,-0.56,1.60,13.35,XY24,NTT,PAINTS
73,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.77,108518.0,-23317.0,65545.0,0.08,-17.69,60.40,32.03,98.0,-0.36,0.77,21.55,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.15,59.0,H-LC,12.58,298196.0,-47760.0,122618.0,0.85,-13.81,41.12,21.64,1.0,-0.39,2.12,5.64,X40,ATH,IT
40,INFY,2275.00,-16.73,55.0,H-LC,8.65,324868.0,11980.0,159705.0,0.92,3.83,49.16,54.87,3.0,0.08,2.31,10.73,X40,BTT,IT
42,ITC,452.00,-37.92,49.0,H-LC,2.59,200392.0,254.0,20640.0,-0.80,0.13,10.30,10.44,4.0,0.01,1.43,5.70,X40,NTT,FMCG
84,VBL,671.64,-15.68,44.0,H-LC,6.05,303688.0,-12154.0,124816.0,2.83,-3.85,41.10,35.67,5.0,-0.10,2.16,9.47,X40N,ATH,FMCG
1,ABB,7934.00,-38.31,64.0,H-LC,10.76,257861.0,-3758.0,122974.0,0.43,-1.44,47.69,45.57,7.0,-0.03,1.84,9.72,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,67.0,M-MC,6.97,229990.0,5028.0,162626.0,0.47,2.24,70.71,74.53,192.0,0.03,1.64,35.21,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,12.39,62.0,H-MC,3.58,193990.0,29320.0,38643.0,0.82,17.81,19.92,41.28,89.0,0.76,1.38,27.21,X40N,ATH,AC
30,HAVELLS,2069.16,1.05,63.0,H-MC,1.86,250583.0,2852.0,72218.0,-0.53,1.15,28.82,30.30,92.0,0.04,1.78,14.60,X40,ATH,ELECTRICAL
85,VOLTAS,1918.49,0.74,57.0,H-MC,2.56,211350.0,19608.0,76424.0,-0.40,10.23,36.16,50.08,99.0,0.26,1.50,17.72,XY25,ATH,AC
18,CAMS,5211.76,-2.28,60.0,H-SC,4.21,111104.0,9100.0,34820.0,2.00,8.92,31.34,43.06,122.0,0.26,0.79,27.56,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,68.0,H-MC,6.31,114226.0,13722.0,16917.0,-0.44,13.65,14.81,30.49,79.0,0.81,0.81,50.48,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-40.16,54.0,M-SC,0.46,101100.0,9602.0,69152.0,-1.01,10.49,68.40,86.07,223.0,0.14,0.72,46.82,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.47,45.0,H-SC,12.00,135558.0,12222.0,121541.0,-1.54,9.91,89.66,108.45,119.0,0.10,0.96,34.48,AR,ATH,MISC
87,WIPRO,420.00,-10.16,58.0,M-LC,5.58,158125.0,7180.0,102275.0,0.41,4.76,64.68,72.51,53.0,0.07,1.13,11.07,XR,NTT,IT
36,IEX,219.00,-28.31,61.0,H-SC,17.10,204744.0,4790.0,92442.0,2.37,2.40,45.15,48.63,115.0,0.05,1.46,14.23,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1905.05,68.0,L-SC,3.41,54720.0,-82457.0,82551.0,1.46,-60.11,150.86,0.07,267.0,-1.00,0.39,36.09,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.57,84518.0,-29031.0,69085.0,-2.76,-25.57,81.74,35.27,268.0,-0.42,0.60,106.86,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-2.99,51.0,H-SC,13.01,90248.0,-10715.0,105247.0,1.17,-10.61,116.62,93.63,148.0,-0.10,0.64,34.42,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,53.0,H-SC,12.96,94800.0,-3180.0,24781.0,-1.14,-3.25,26.14,22.05,152.0,-0.13,0.67,37.80,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,68.0,H-MC,6.31,114226.0,13722.0,16917.0,-0.44,13.65,14.81,30.49,79.0,0.81,0.81,50.48,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3856.76,63.0,M-SC,6.15,123357.0,-24011.0,75828.0,-1.36,-16.29,61.47,35.16,236.0,-0.32,0.88,26.00,XY24,NTT,AUTO
54,RAJESHEXPO,518.0,1905.05,68.0,L-SC,3.41,54720.0,-82457.0,82551.0,1.46,-60.11,150.86,0.07,267.0,-1.00,0.39,36.09,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.0,-43.66,51.0,L-SC,42.57,84518.0,-29031.0,69085.0,-2.76,-25.57,81.74,35.27,268.0,-0.42,0.60,106.86,XR,NTT,HOTELS
27,GREENPANEL,537.0,237.86,66.0,M-SC,3.46,150963.0,-32115.0,111094.0,0.95,-17.54,73.59,43.14,230.0,-0.29,1.07,42.00,XY24,NTT,MISC
76,TITAGARH,1548.0,0.74,68.0,H-SC,3.42,165086.0,-26929.0,105820.0,0.49,-14.02,64.10,41.08,158.0,-0.25,1.18,39.89,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.52
1,25,42.82
2,50,73.24


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.28
LC    34.06
MC    22.66
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.49
X40      15.00
X40N     12.85
XR       11.42
XY25      9.58
AR        8.52
OX40N     8.02
X200      1.74
MH        1.68
X5K       1.43
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.46
H-LC    25.90
H-MC    19.75
M-SC    13.30
M-LC     7.11
M-MC     2.61
L-SC     1.52
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.23,-13.15,74.84
FMCG,11.58,-4.05,40.45
FINANCE,9.48,-15.44,60.65
BANKS,7.67,-11.16,62.34
MISC,7.34,-11.50,73.51
PAINTS,5.71,-15.11,32.27
ELECTRICAL,5.61,-4.53,62.15
AC,3.58,3.77,28.82
INSURANCE,3.34,-7.16,43.91


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2843480.0,22
XR,1306335.0,14
AR,1299876.0,9
X40,742303.0,10
X40N,607080.0,10
OX40N,519377.0,10
XY25,404225.0,7
SR,243372.0,2
X5K,117112.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3415092.0,29
M-SC,1377803.0,17
H-LC,1236401.0,15
H-MC,1176234.0,15
M-LC,380642.0,5
M-MC,312116.0,2
L-SC,248058.0,3
L-MC,60397.0,1
L-LC,36707.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1030282.0      6
M-SC       XY24       797848.0      7
H-SC       AR         784596.0      5
           XR         754841.0      7
H-MC       XY24       487060.0      4
H-LC       X40        461652.0      5
           AR         353973.0      2
M-MC       XY24       312116.0      2
H-SC       X40N       290484.0      4
           OX40N      249515.0      4
           SR         243372.0      2
H-LC       X40N       218382.0      4
H-MC       X40        211465.0      4
           XY25       187278.0      2
L-SC       XR         165507.0      2
M-SC       AR         161307.0      2
M-LC       XY24       156070.0      2
M-SC       OX40N      121766.0      4
M-LC       X5K        117112.0      1
M-SC       XY25       114136.0      1
           XR         113560.0      2
H-MC       XR         109755.0      1
M-LC       XR         102275.0      1
H-MC       X40N        98214.0      2
L-SC       OX40N       82551.0      1
H-LC       X200        81371.0      1
M-SC       X40         69186.0      1
H-MC       OX40N       65545.0      1
H-SC       MH          62002.0      1
H-LC       XY25        60919.0      2
L-MC       XR          60397.0      1
H-LC       XY24        60104.0      1
L-LC       XY25        36707.0      1
H-MC       MH          16917.0      1
M-LC       XY25         5185.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
